### import dependencies

In [9]:
##### work in this repository was inspired by 
##### https://github.com/Kazuhito00/hand-gesture-recognition-using-mediapipe/blob/main/README_EN.md
##### https://www.youtube.com/watch?v=a99p_fAr6e4


### import dependencies

In [10]:
import csv
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
import pandas as pd


# this is just to make sure that the results are reproducible to anyone that runs the code lol.
RANDOM_SEED = 42

#### data formatting
https://devqa.io/python-convert-csv-file-to-list/

In [11]:
import glob
labels = {0: "no_yawn", 1: "yawn"}

Y_train = [] #1d array, (big_number, 1)
X_train = [] #2d array, (big_number, 42)
def data_append_train(data, X_train = X_train, Y_train = Y_train):
  #formatting the labels, although I am not so sure what is required...
  data1 = "data/train"
  for label in labels: 
    name = labels[label]
    path = f"{data1}/{name}/*.csv"
    print(name)

    # getting the directory
    files = glob.glob(path)
    print(files)
    print(len(files))

    # now concatenating the content of the files.
    for f in files:
        with open(f, 'r') as file:
            csv_reader = csv.reader(file)
        
            i = 0
            for row in csv_reader:
                #skip the header line
                if i == 0: 
                    i += 1
                    continue
                
                X_train.append([float(num) for num in row])
                Y_train.append(label)
    
    print(X_train[-1])
    print("training set:")
    print(len(X_train))
    print(len(Y_train))    

Y_test = []
X_test = []
def data_append_test(data, X_test = X_test, Y_test = Y_test):
  #formatting the labels, although I am not so sure what is required...
  data1 = "data/test"
  for label in labels: 
    name = labels[label]
    path = f"{data1}/{name}/*.csv"
    print(name)

    # getting the directory
    files = glob.glob(path)
    print(files)
    print(len(files))

    # now concatenating the content of the files.
    for f in files:
        with open(f, 'r') as file:
            csv_reader = csv.reader(file)
        
            i = 0
            for row in csv_reader:
                #skip the header line
                if i == 0: 
                    i += 1
                    continue
                
                X_test.append([float(num) for num in row])
                Y_test.append(label)
    
    print(X_test[-1])
    print("training set:")
    print(len(X_test))
    print(len(Y_test))    
#i'll only use the train test.
data = "data"
data_append_train(data)
data_append_test(data)




no_yawn
['data/train/no_yawn\\eye2.csv']
1
[222.25069046020508, 233.9298677444458, -36.499876976013184, 222.02556610107422, 225.97487926483154, -38.721935749053955, 221.22488021850586, 215.78165531158447, -35.770583152770996, 218.77561569213867, 191.43005847930908, -16.770299673080444, 217.38073348999023, 173.00106525421143, -11.76937460899353, 222.51628875732422, 268.6786937713623, -14.411613941192627, 216.42608642578125, 234.080171585083, -36.190924644470215, 185.33504486083984, 271.6083526611328, -1.4509478211402893, 148.03768157958984, 269.67607498168945, 57.80587196350098, 204.4702911376953, 237.68178462982178, -17.349413633346558, 198.54713439941406, 237.12430000305176, -16.496912240982056, 199.32052612304688, 272.8155040740967, -7.263230085372925, 215.9865951538086, 268.97552490234375, -14.164596796035767, 205.2019500732422, 270.32618522644043, -9.641178846359253, 204.17892456054688, 271.6710376739502, -10.759450197219849, 137.58285522460938, 232.24735736846924, 68.7466478347778

### since I already have the training dataset split, I just need to train!

In [12]:
# X_df_train = pd.DataFrame(X_train)
# Y_df_train = pd.DataFrame(Y_train)

# X_df_test = pd.DataFrame(X_test)
# Y_df_test = pd.DataFrame(Y_test)


X_train = np.asarray(X_train)
Y_train = np.asarray(Y_train)
# 
X_test = np.asarray(X_test)
Y_test = np.asarray(Y_test)

from sklearn.utils import shuffle

# the random_state is so that both shuffle perform the same.
X_train,Y_train = shuffle(X_train, Y_train, random_state=0)
X_test,Y_test = shuffle(X_test, Y_test, random_state=0)
# X_train.shape, Y_train.shape
X_train, Y_train


(array([[342.18013763, 260.15885353, -37.74030209, ..., 352.32494354,
         260.89742661, -32.1662879 ],
        [329.57881927, 205.42036057, -44.47785854, ..., 342.2070694 ,
         203.77053738, -39.3476367 ],
        [261.48199081, 217.17184067, -30.72162867, ..., 269.2628479 ,
         215.71579456, -25.85042715],
        ...,
        [189.49829102, 237.83334732, -58.42456341, ..., 208.16854477,
         233.92504692, -54.99604702],
        [399.60113525, 239.40487862, -51.03535652, ..., 415.53142548,
         238.31093788, -46.16221905],
        [366.90582275, 233.97930622, -44.0205574 , ..., 379.80670929,
         232.25850105, -38.63370895]]),
 array([0, 0, 1, ..., 0, 1, 1]))

In [13]:
NUM_CLASSES = 2 
# model_save_path = "model/"
from tensorflow.keras.callbacks import ModelCheckpoint
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    "model/yawn_model/model.{epoch:02d}-{val_accuracy:.2f}",
    monitor='val_accuracy',
    verbose=0,
    save_best_only=True,
    save_weights_only=False,
    mode='auto',
    #everytime the accuracy gets better, it saves
    save_freq='epoch',
)


#defining the model 

In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
import keras

In [15]:
print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)


Shape of X_train: (3308, 174)
Shape of X_test: (700, 174)


In [23]:

model = Sequential([
    Dense(units=32, activation='relu', input_shape=(174,)),
    Dense(units=64, activation='relu',
        kernel_regularizer=keras.regularizers.l1_l2(0.05)),
    Dense(units=256, activation='relu'),
    Dropout(0.2),
    Dense(units=128, activation='relu'),
    Dense(units=64, activation='relu'),
    Dense(units=16, activation='relu', 
        kernel_regularizer=keras.regularizers.l1_l2(0.01)),
    Dropout(0.2),
    # Dense(units=1, activation='sigmoid')
    Dense(units = NUM_CLASSES, activation = "softmax")
])

In [24]:
#cost function
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam

# model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=["accuracy"])
model.compile(loss=SparseCategoricalCrossentropy(), optimizer=Adam(), metrics=["accuracy"])

In [25]:
#fitting
history = model.fit(X_train,Y_train, epochs=150, batch_size=32, validation_data=(X_test,Y_test), callbacks=[checkpoint])

Epoch 1/150
104/104 [==============================] - 3s 12ms/step - loss: 13.1171 - accuracy: 0.5320 - val_loss: 9.4841 - val_accuracy: 0.4829
Epoch 2/150
104/104 [==============================] - 1s 7ms/step - loss: 7.4761 - accuracy: 0.5632 - val_loss: 5.9312 - val_accuracy: 0.4829
Epoch 3/150
104/104 [==============================] - 1s 8ms/step - loss: 5.1162 - accuracy: 0.5626 - val_loss: 4.6566 - val_accuracy: 0.4829
Epoch 4/150
104/104 [==============================] - 1s 7ms/step - loss: 4.2229 - accuracy: 0.5629 - val_loss: 3.9046 - val_accuracy: 0.4457
Epoch 5/150
104/104 [==============================] - 1s 9ms/step - loss: 3.5264 - accuracy: 0.6291 - val_loss: 3.4313 - val_accuracy: 0.6357
Epoch 6/150
104/104 [==============================] - 1s 7ms/step - loss: 3.0443 - accuracy: 0.6091 - val_loss: 2.8201 - val_accuracy: 0.6571
Epoch 7/150
104/104 [==============================] - 1s 7ms/step - loss: 2.5871 - accuracy: 0.5735 - val_loss: 2.4536 - val_accuracy: 0.48